In [1]:
from binance.client import Client
import pandas as pd
import json
import numpy as np
# from sklearn.linear_model import LinearRegression

In [2]:
from security import get_keys
binance_key = get_keys('client')
client = Client(binance_key['api_key'], binance_key['api_secret'])

correct password


In [46]:
float(client.get_asset_balance('BTC',recvWindow=10000)['free'])

0.00433

In [3]:
def get_total_asset(coin_list, base_coin):
    total_asset = 0.0
    for coin_symbol in coin_list:
        pair = coin_symbol + base_coin
        if coin_symbol == base_coin:
            coin_asset = float(client.get_asset_balance(coin_symbol,recvWindow=10000)['free'])
            total_asset += coin_asset
            print(coin_symbol, coin_asset)
            continue
        coin_asset = float(client.get_asset_balance(coin_symbol,recvWindow=10000)['free']) * float(client.get_symbol_ticker(symbol=pair)['price'])
        total_asset += coin_asset
        print(coin_symbol, coin_asset)
    return total_asset

In [6]:
initial_asset = 48.26

In [8]:
current_asset = get_total_asset(coin_list = ['BTC', 'BNB', 'USDT'], base_coin = 'USDT')
gain = (current_asset/initial_asset) - 1.0
gain *= 100.0
display =  ("-" if gain < 0 else "+") + "{0:.4f} %".format(abs(gain))
display

BTC 0.0
BNB 0.6305848659359999
USDT 47.47751145


&#39;-0.3148 %&#39;

In [ ]:
data = pd.read_csv("BTCUSDT.csv", header=None, usecols=[2,8])
data = data.rename(columns={2: "date", 8: "price"})
data.date = pd.to_datetime(data.date, unit='ms')
data

In [ ]:
csvdata = pd.read_csv("BTCUSDT.csv", header=None, usecols=[2,8])
csvdata = csvdata.rename(columns={2: "date", 8: "price"})
csvdata.date = pd.to_datetime(csvdata.date, unit='ms')

In [ ]:
price = {'BTCUSDT': csvdata, 'error':False}

In [ ]:
def MA(sec):
	df = price['BTCUSDT']
	start_time = df.date.iloc[-1] - pd.Timedelta(seconds=sec)
	df = df.loc[df.date >= start_time]
	return df.price.mean()

In [ ]:
for x in range(10):
    print(MA(x*30))

In [ ]:
x = MA(30) / MA(900)
x -= 1.0
x *= 100.0
print(x, '%')

In [ ]:
def buyAmount(coin, pair):
    balanceBuy = float(client.get_asset_balance(coin,
    recvWindow=10000)['free'])
    close = float(client.get_symbol_ticker(symbol=pair)['price'])
    maxBuy = round(balanceBuy / close * .995, 5)
    return maxBuy
    
def sellAmount(coin):
    balanceSell = float(client.get_asset_balance(coin,
    recvWindow=10000)['free'])
    maxSell = round(balanceSell * .995, 5)
    return maxSell

In [ ]:
buyAmount('USDT', 'BTCUSDT')

In [ ]:
sellAmount('BTC')

In [ ]:
maxbuy = buyAmount('USDT', 'BTCUSDT')
client.create_test_order(symbol='BTCUSDT', side='BUY', type='MARKET', quantity=0.00434)

In [ ]:
maxsell = sellAmount('BTC')
client.create_test_order(symbol='BTCUSDT', side='SELL', type='MARKET', quantity=0.00434)

In [ ]:
maxbuy = buyAmount('USDT', 'BTCUSDT')
client.create_order(symbol='BTCUSDT', side='BUY', type='MARKET', quantity=0.00433)

In [ ]:
maxsell = sellAmount('BTC')
client.create_order(symbol='BTCUSDT', side='SELL', type='MARKET', quantity=0.00433)